In [378]:
import glob
import ipywidgets
import ipyvolume
import cv2
import numpy as np
import os
import time

In [379]:
import pythreejs
figcam = pythreejs.PerspectiveCamera(aspect=0.8, fov=46.0, matrixWorldNeedsUpdate=True, position=(1.785528946985704, 1.3757523415625494, 2.412892214337929), projectionMatrix=(2.9448154572796907, 0.0, 0.0, 0.0, 0.0, 2.3558523658237527, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion=(-0.19796316984919116, 0.30892794387643674, 0.08046673488216134, 0.9267681551784899), rotation=(-0.5181948251143776, 0.5713287660544955, 0.3285944197012996, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(-0.27146072492949946, 0.9086713759213286, -0.31721507751364897))

In [380]:
figcam = pythreejs.PerspectiveCamera(aspect= 0.8, fov= 46.0, matrixWorldNeedsUpdate= True, position= (1.8634035771785555, 1.8424395267808371, 1.247399960480493),  projectionMatrix= (2.9448154572796907, 0.0, 0.0, 0.0, 0.0, 2.3558523658237527, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion= (-0.30349208511373676, 0.4393038577522591, 0.14736527540169084, 0.8325792157164885), rotation= (-0.9756484969113725, 0.6971850344672847, 0.7313655980811482, 'XYZ'), scale= (1.0, 1.0, 1.0),  up=(-0.5120370184110512, 0.7723520597582004, -0.3758861364349397))

In [381]:
  figcam = pythreejs.PerspectiveCamera(aspect= 0.8, fov= 46.0, matrixWorldNeedsUpdate= True, position= (1.863950351022152, 0.9868301900448053, 0.7457858863923755),  projectionMatrix= (2.9448154572796907, 0.0, 0.0, 0.0, 0.0, 2.3558523658237527, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion= (-0.19995936160138095, 0.5415956146940057, 0.10846490922136856, 0.8092748651229632), rotation= (-0.9236324771770482, 0.9849099252837857, 0.7883790508844056, 'XYZ'), scale= (1.0, 1.0, 1.0),  up=(-0.3921500762820368, 0.8965032343511368, -0.20615593241510943))

## Volume Rendering

In [412]:
levels = {
    'ucsdLED2sm':
         {
             'SR': { '1k': [0.18, 0.00], '2k': [0.18, 0.00], '4k': [0.15, 0.00] },
             'LR': { '1k': [0.55, 0.00], '2k': [0.45, 0.00], '4k': [0.35, 0.00] },
             'HR': [1.00, 0.75]
         }
}

opacities = {
       'ucsdLED2sm':
            { 'SR': { '1k': [0.20,0.00,0.00], '2k': [0.20,0.00,0.00], '4k': [0.20,0.00,0.00]},
              'LR': { '1k': [0.15,0.00,0.00], '2k': [0.15,0.00,0.00], '4k': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
}

In [383]:
fig_res = {} # dictionary of result figures
fig_inp = {} # dictionary of input figures

In [384]:
sample = 'ucsdLED2sm'
noises = list(levels[sample].keys())
#frame_range = [200,509,0,509]
frame_range = [0,10,60,360,10,125]
#frame_range = [0,512,0,512]
#frame_range = [0,390,5,58]
#extent = [[0, 0.3], [0, 1.0], [0, 0.2]]
extent = np.array([[0, 0.1], [0, 0.5], [0, 1]])
transpose_arr = [1, 0, 2]
extent =  extent[transpose_arr]
#extent = np.array([[0, 1.0], [0, 0.4], [0, 0.1]])
opacity = 5.00
brightness = 2.00

In [385]:
noise = '4k'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#input_vol = input_vol[:,::-1,::-1]
input_vol = input_vol[:,:,::-1]
input_vol =  np.transpose(input_vol, transpose_arr)
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],extent=list(extent),level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [410]:
noise = '4k'
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [386]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#avg_img = avg_img[:,::-1,::-1]
avg_img = avg_img[:,:,::-1]
avg_img =  np.transpose(avg_img, transpose_arr)
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],extent=list(extent),level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
#ipyvolume.selector_default()
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [411]:
noise = '4k'
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [387]:
noise = '2k'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#input_vol = input_vol[:,::-1,::-1]
input_vol = input_vol[:,:,::-1]
input_vol =  np.transpose(input_vol, transpose_arr)
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],extent=list(extent),level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [408]:
noise = '2k'
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [388]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#avg_img = avg_img[:,::-1,::-1]
avg_img = avg_img[:,:,::-1]
avg_img =  np.transpose(avg_img, transpose_arr)
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],extent=list(extent),level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
#ipyvolume.selector_default()
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [407]:
noise = '2k'
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [389]:
noise = '1k'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#input_vol = input_vol[:,::-1,::-1]
input_vol = input_vol[:,:,::-1]
input_vol =  np.transpose(input_vol, transpose_arr)
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],extent=list(extent),level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [406]:
noise = '1k'
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [390]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#avg_img = avg_img[:,::-1,::-1]
avg_img = avg_img[:,:,::-1]
avg_img =  np.transpose(avg_img, transpose_arr)
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],extent=list(extent),level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
#ipyvolume.selector_default()
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [405]:
noise = '1k'
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [391]:
gt_vol = np.load('result_vol_npys/target_'+sample+'_vol.npy')
gt_vol = gt_vol[frame_range[0]:frame_range[1],frame_range[2]:frame_range[3],frame_range[4]:frame_range[5]]
#gt_vol = avg_img[:,::-1,::-1]
gt_vol = gt_vol[:,:,::-1]
gt_vol =  np.transpose(gt_vol, transpose_arr)
mins = gt_vol.min(axis=(0,1))
maxs = gt_vol.max(axis=(0,1))
gt_vol_norm = (gt_vol - mins) / (maxs - mins)
fig_gt = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(gt_vol_norm[:,:,:],extent=list(extent),level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise],specular_exponent=5)
fig_gt.volumes[0].opacity_scale = opacity
fig_gt.volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.style.box_off()
ipyvolume.show()

In [392]:
ipyvolume.pylab.savefig('gt_3dfig_'+sample+'.png',width=1280,height=720,fig = fig_gt)

Output()

## Convert full UCSD sample data into MATLAB

In [399]:
import scipy.io as sio

In [403]:
noises = ['1k','2k','4k']
for noise in noises:
    sr_data = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
    lr_data = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
    mins = sr_data.min(axis=(0,1))
    maxs = sr_data.max(axis=(0,1))
    sr_data_norm = (sr_data - mins) / (maxs - mins)
    mins = lr_data.min(axis=(0,1))
    maxs = lr_data.max(axis=(0,1))
    lr_data_norm = (lr_data - mins) / (maxs - mins)    
    sio.savemat(sample+'_'+noise+'_LR.mat',{'vect':lr_data})
    sio.savemat(sample+'_'+noise+'_SR.mat',{'vect':sr_data})
    sio.savemat(sample+'_norm_'+noise+'_LR.mat',{'vect':lr_data_norm})
    sio.savemat(sample+'_norm_'+noise+'_SR.mat',{'vect':sr_data_norm})
hr_data = np.load('result_vol_npys/target_'+sample+'_vol.npy')
mins = hr_data.min(axis=(0,1))
maxs = hr_data.max(axis=(0,1))
hr_data_norm = (hr_data - mins) / (maxs - mins) 
sio.savemat(sample+'_HR.mat',{'vect':hr_data})
sio.savemat(sample+'_norm_HR.mat',{'vect':hr_data_norm})